# Missouri East Cast Iron Model Phase 1
This is the first script for the mandated cast iron model replacement. It is designed to modify the cast iron model points by taking in an input CSV and adding leak repairs & pipe observations that are within 10' from cast iron main to the master point feature class.

After it is run, a manual process will be needed to move some points closer and, occasionally, to generate points that were unable to be joined with an existing leak repair or pipe observation point.

## Procedure to run Cast Iron Model (Missouri East)


### Manual Editing
This step needs to be accomplished after this script is run.  

Edit the MoveCloser feature class. It will have a number of point features that are further than 10' away from the nearest main. You will want to move those points onto the nearest main. If the point is out in the middle of nowhere, you can just delete it. 

This is also an opportunity to clean up existing points and main segment issues since editing is already in progress. The existing points are named *CIModelPoints* and the existing cast iron main segments are called *CI_MainModelSegments*. 

#### Check the Error_UnfoundPoints feature class
If this first script finds missing records from the input table then it will generate a new, empty, feature class called *Error_UnfoundPoints*. You will need to manually transfer any missing records to this feature class. The field names should match up with the input data's csv/excel file except spaces will be filled with a "_" symbol. 

Make sure to fill out the following fields: *Actual_Finish, Primary_Break_Cause, Cast_Iron_Evaluation, Replacement_Criteria_Met, Wall_Remaining*

### Script Inputs
This script has several inputs, which are the various data pieces that are entered into it in order for it to run. This section goes over those, calling out the variable names, so that they can be modified as needed.
The variables *ws* and *ws_gdb* are the first variables called. Variable ws holds the location to a folder directory and stands for workspace. It is where all of the outputs will be written to as well as where the workspace geodatabase will need to be located at. The Variable *ws_gdb* calls upon the *ws* variable and then assigns a geodatabase name within that folder. If you wish to redirect the script to use data in a different directory and geodatabase, you have to change these variables.

The *ws_gdb* geodatabase is expected to hold the "master" model data which is referenced in the variables *ci_masterpoints*. The point feature class this  variable refences MUST be set up with specific field names. That is because the input CSV that comes from the Maximo9 side contains certain field names. The field names of the CSV, when converted into a table, need to match what is in the *ci_masterpoints*. 

The *in_csv* variable contains the input location for the input csv. This is used to translate that csv into a arcGIS table in the *ws_gdb* geodatabase. *in_csv* can be changed each time the script is run with no specific requirements or issues. If there are extra fields within the csv, the script will ignore them.

*LeakRepairMaximo* and *PipeOb* are variables that refences feature services. This is because the Notebook Server can not reference SDE connection paths nor UNC folder paths easily. These should not be changed.

## Python Script
These cells contain the scripting cells in Python for conducting the mandated model.

### Import modules and Setup Functions
The first step of the script is to import needed modules and set up any functions needed for the script. This script uses one function, *cleanMe*, to help clean up data prior to doing an append. This section of code should not need to be modified. 

In [ ]:
# import modules
import arcpy, os, datetime

def cleanMe(layer, fldname, fld_list, del_field):
    """ Cleans a layer after a join
    Input requires an input feature class or table, the name of that table,
    a list of fields to be deleted, and the join-name to identify fields to delete.
    
    This function works to clean up a join to delete every field that had been
    joined. The joins it is intended to work on do not need fields from the join
    but rather just location data and are attaching a table to a feature class.
    """
    # Rename the deletion list based on the input field name
    # deleting easier for fields that may have a duplicate after renaming like OBJECTID
    del_list = [(fldname + "_" + item) for item in fld_list]
    # Iterate through all fields in leak repair list that have the CI csv table name
    # in the field name.
    for field in arcpy.ListFields(layer):
        if not field.required:
            # Identify if the field name is in the deletion list
            if field.name in del_list:
                #...if it is, delete it
                arcpy.management.DeleteField(layer, field.name)
                #Else if the field name has MXSPAT_LeakRepair
            elif del_field in field.name:
                # delete the field
                arcpy.management.DeleteField(layer, field.name)
            # Else, alter fields
            else: 
                # ...then get a new name removing the name of the CI csv table
                newName = str(field.name.replace(fldname + "_", ""))    
                #.....and change the name
                arcpy.management.AlterField(layer, field.name, newName)

### Environment Settings and Workspace Variables
Next, set up environment settings and variable names. When you want to utilize a new workspace for the model, you can change the variables *ws* and *ws_gdb* to the name of the folder and the geodatabase respectively. *CastIronMainSegments* defines the name used for the cast iron main segments created previously while *ci_masterpoints* points to the point feature class containing all current cast-iron model points.

In [ ]:
# set environment settings
arcpy.env.overwriteOutput=True
# Set feature class workspace
ws = '/arcgis/directories/'
# Set gdb workspace
ws_gdb = os.path.join(ws, "data.gdb")
# Set layer variables
# Master cast iron point layer (updated in this script)
ci_masterpoints = os.path.join(ws_gdb, "CIModelPoints")
# set variables using the feature services
LeakRepairMaximo = "Portal REST Service"
PipeOb = "Portal REST Service"

### Convert CSV into GDB Table
Below is code to convert the input csv table into an arcGIS table inside the workspace geodatabase. The variable *in_csv* can be changed to point to the input data each time the script is run. The cell below will also output a print of how many rows are in the created table. Verify that the table was created properly by checking that count against the number of rows in your input data. The counts should match.

In [ ]:
# Convert the CSV to a table
in_csv = 'location/file.csv'
ciTable_name = "CIModel"
ci_table = arcpy.conversion.TableToTable(in_csv, ws_gdb, ciTable_name)
# Get count of all rows in the imported CSV. Check this against the excel document
# to verify they are the same
csv_count = int(arcpy.management.GetCount(ci_table).getOutput(0))
print("The imported table named {0} has a total of {1} rows.".format(ciTable_name, csv_count))

### Create New Leak Repairs
Run the below cell to create the *new_lr* feature class which will contain only the leak repairs identified in the input csv. It will output a statement letting you know where the new leak repairs were generated and how many were found. This will likely be slightly smaller than the count from the previous step.

In [ ]:
# Convert the CI table work orders into a text field
wo_txtfld = "MXWONUM"
arcpy.AddField_management(ci_table, wo_txtfld, 'TEXT')
arcpy.CalculateField_management(ci_table, wo_txtfld, '!Work_Order_ID!', 'PYTHON')
#Create table view and feature layer to allow join
ci_view = arcpy.MakeTableView_management (ci_table, ciTable_name)
lr_lyr = arcpy.MakeFeatureLayer_management(LeakRepairMaximo, "LRMaximo_Lyr")
# Add join using the work order id field
arcpy.AddJoin_management (lr_lyr, wo_txtfld, ci_view, wo_txtfld, 'KEEP_COMMON')
# Set path for new leak repairs
new_lr_name = "newLeakRepairs"
new_lr = os.path.join(ws_gdb, new_lr_name)
# Copy those leak repairs to feature class
arcpy.CopyFeatures_management (lr_lyr, new_lr)
lr_count = int(arcpy.management.GetCount(new_lr).getOutput(0))
print("Copied new leak repairs to {0}. A total of {1} leak repairs were found.".format(new_lr, lr_count))

### Check for Duplicates
After the new leak repair feature class is generated, we need to check for duplicate records. Sometimes there are leak repairs that have been updated. This is done by checking the location of new leak repairs to the master model feature class. The below code will output a count of the number of duplicates found that had to be removed. It will also create a copy of the master points named in the *date_master* variable. This copy contains the data prior to anything being deleted.

In [ ]:
## Clean up the field names of layers in the joined leak repair list
# Create list of field names to delete from data after cleaning it
del_list = ['OBJECTID', 'Work_Order_ID', 'GLOBALID']
join_fc_name = "L1Leak_Repair"
cleanMe(new_lr, ciTable_name, del_list, join_fc_name)
# Remove the join
arcpy.RemoveJoin_management (lr_lyr)
# Create view layer of master ci points data 
master_lyr = arcpy.MakeFeatureLayer_management (ci_masterpoints, 'CIModelPointsLyr')
# Create feature layer of leak repairs that exist in the model
newlr_lyr = arcpy.MakeFeatureLayer_management (new_lr, new_lr_name)
# Prior to selection and deletion, copy of master layer with date for later use or deletion
date = str(datetime.datetime.now().date()).replace("-", "_")
date_master = "CIModelPoints" + "_" + date
arcpy.CopyFeatures_management(master_lyr, os.path.join(ws_gdb, date_master))

# Select existing cast iron lr points that match the leak repairs that exist in the model
arcpy.SelectLayerByLocation_management (master_lyr, 'ARE_IDENTICAL_TO',newlr_lyr)
dupe_count = int(arcpy.management.GetCount(master_lyr).getOutput(0))
# Delete features in Ci Model points layer that match those found in the previous selection
# this is done to clean up previously existing leak repairs that may have an update to their data
arcpy.DeleteFeatures_management(master_lyr)
print("Deleting {0} features from the master model points that were found to be duplicates. These are likely records that have updates to be applied.".format(dupe_count))

### Append Leak Repairs within 10'
The below cell appends leak repairs that are within 10' of cast iron main to the master point feature class. It will print out a count of how many records were found within 10'. Afterwards, the code will generate the *move_lr* feature class with points that were further than 10'. It will give you a count of how many of those were found. 

The string in *move_lr* can be changed to change the name of the output feature class in the workspace gdb created previously (*ws_gdb*). 

These found points will be the ones to be moved manually.

In [ ]:
# Cast iron main segements (Created in script 2)
CastIronMainSegments = os.path.join(ws_gdb, "CI_MainModelSegments")
# Create feature layer for cast iron main segments
ci_main_lyr = arcpy.MakeFeatureLayer_management (CastIronMainSegments, 
                                                 'CastIronMainSegmentsLyr')
# Find new/updated leak repairs within 10' of cast iron 
arcpy.SelectLayerByLocation_management (newlr_lyr, 'WITHIN_A_DISTANCE',ci_main_lyr, 
                                        '10 FEET')
lrTenFt_count = int(arcpy.management.GetCount(newlr_lyr).getOutput(0))
# Append the leak repairs that are within 10' of cast iron main to the master point list
arcpy.Append_management(newlr_lyr, master_lyr,"NO_TEST")
print("A total of {0} records were found that are within 10' of a cast iron main.".format(lrTenFt_count))
# Select Leak repair layer and swap the selection to find points further than 10' away
arcpy.SelectLayerByAttribute_management(newlr_lyr,"SWITCH_SELECTION","#")
closer_count = int(arcpy.management.GetCount(newlr_lyr).getOutput(0))
# Create a feature class of points that need to be manually moved onto cast iron pipe
move_lr = os.path.join(ws_gdb, "MoveCloser")
arcpy.management.CopyFeatures(newlr_lyr, move_lr)
print("A total of {0} records were found that are further than 10' of a cast iron main and will need to be manually moved closer.".format(closer_count))
#Clean up feature layers
for item in [newlr_lyr, ci_main_lyr]:
    arcpy.management.Delete(item)

### Check Counts
Run the below to check record counts. This is to verify that everything adds up appropriately and no leak repairs are being missed. If the count is off, the code segment will output an error and will generate a new feature class named in the *error_fc* variable. To change the name of the created, blank, feature class, change the string assigned to that variable. 

The empty feature class can be used to add the missed records.

In [ ]:
# Checking Leak Repair Counts
print("Checking leak repair counts...")
print("Total new records (Leak Repairs and Pipe Observations) in input table: {0}.".format(csv_count))
print("Total number of new Leak Repairs identified: {0}.".format(lr_count))
print("New Leak Repairs 10' to cast iron main that have been appended: {0}".format(lrTenFt_count))
print("New Leak Repairs further than 10' from cast iron main: {0}.".format(closer_count))
if (closer_count + lrTenFt_count) == lr_count:
    print("The counts for new leak repairs within 10' and further than 10' match the number of new leak repairs found.")
else: 
    print("The count for the new leak repairs is off. There are some records not being counted as within 10' or further than 10'. Check the data for problems.")
    error_fc = "Error_UnfoundPoints"
    arcpy.management.CreateFeatureclass(ws_gdb, error_fc, "POINT", ci_view)
    print("An empty feature class named {0} has been created. Please move records that did not get added in the above steps to this feature class.".format(error_fc))

### Identify new pipe observations
The below cell is used to identify pipe observations and create a table of them.

In [ ]:
#### Identify new pipe observations##
# Join leak repair layer to new CI table
arcpy.AddJoin_management (ci_view, 'MXWONUM',lr_lyr, 'MXWONUM', 'KEEP_ALL')
# sql query needed to identify null data
query=r"L1Leak_Repair.OBJECTID IS null"
# Select new records that aren't in leak repair data and are thus pipe observations
arcpy.SelectLayerByAttribute_management(ci_view,"",query)
# Clean up join to verify field names aren't crazy
arcpy.RemoveJoin_management (ci_view)
# Copy your new pipe obs records to table
# Can probably create this "memory\"
new_pipeobs_name = "PipeObsTbl"
new_obs = os.path.join(ws_gdb, new_pipeobs_name)
#new_obs = os.path.join(ws_gdb, r"memory\" + new_pipeobs_name)
arcpy.CopyRows_management (ci_view, new_obs)
# Count new observations
newpo_count = int(arcpy.management.GetCount(new_obs).getOutput(0))
# Clean up view and feature layers
for item in [ci_view, lr_lyr]:
    arcpy.management.Delete(item)

### Check Pipe Observation Counts
Check counts to see if the total of new pipe observations and leak repairs added matches the count from the input csv.

In [ ]:
# Check Pipe observation counts
print("A total of {0} new pipe observations were found.".format(newpo_count))
if newpo_count + lr_count == csv_count:
    print("The total of pipe observations({0}) and leak repairs ({1}) equals the total records found in new input table ({2})."
          .format(newpo_count, lr_count, csv_count))
else: 
    print("The total of pipe observations({0}) and leak repairs ({1}) does not match the total records found in new input table ({2}). Check the data to identify problems."
          .format(newpo_count, lr_count, csv_count))

### Create & Append New Pipe Observations
This section creates the new pipe observations as *pipeObsNewLR*, cleans the fields up using the *cleanMe* function and then appends them to the master data. 

In [ ]:
#### Adding new pipe observations to existing##
arcpy.env.qualifiedFieldNames = False
# Create view table of newly found pipe observations
obs_lyr = arcpy.MakeTableView_management (new_obs, new_pipeobs_name)
# Create feature layer of existing pipe observations from maximo
pipe_name = 'Maximo_PipeObservations'
pipe_lyr = arcpy.MakeFeatureLayer_management (PipeOb, pipe_name)
# Join newly created table from cast iron model to all pipe observations
# second join...not sure if it would work with selection + first join from above if avoiding creating new table
arcpy.AddJoin_management (pipe_lyr, 'WORKORDERMX', obs_lyr, wo_txtfld, 'KEEP_COMMON')
# querty for no nulls
query= "NOT " + new_pipeobs_name + ".OBJECTID IS null"
# Identify where pipeOb table is found
arcpy.SelectLayerByAttribute_management (pipe_name, '', query)
# Copy just id'd pipe obs to FC
pipeObsNewLR = os.path.join(ws_gdb, "pipeObsNewLR")
arcpy.CopyFeatures_management (pipe_lyr, pipeObsNewLR)
# Cleaning up new pipe observations
cleanMe(pipeObsNewLR, new_pipeobs_name, del_list, join_fc_name)
# Append new pipe obervations to master points
arcpy.Append_management(pipeObsNewLR, master_lyr, "NO_TEST")
# Clean up view tables and feature layers and intermediary feature classes/tables
for item in [pipe_lyr, obs_lyr, master_lyr, new_obs, pipeObsNewLR, new_lr]:
    arcpy.management.Delete(item)
# Instead of deleting ci_table and new_lr, change its name to have current date stapled on.
# It is kept due to possible errors in field names next time model is run.
# Set new table name
date_citable = ciTable_name + "_" + date
# Check if table exists prior to rename. If it does, delete. only an issue if the script is run multiple times in the same day
if arcpy.Exists(os.path.join(ws_gdb, date_citable)):
    arcpy.Delete_management(os.path.join(ws_gdb, date_citable))  
# Rename the feature class
arcpy.management.Rename(ci_table, date_citable)